In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- Configuration ---
# IMPORTANT: Replace these with your actual LimeSurvey credentials and domain.
# For better security, consider loading these from environment variables or a secure configuration system
# rather than hardcoding them directly in the script.

# --- Placeholder values - Replace these with your actual details ---
# You can uncomment the os.getenv lines above and set these as environment variables
# in your system before running the script for better security.
LIMEQUERY_DOMAIN = "https://XXX.limequery.com/admin/authentication/sa/login"    # <--- REPLACE THIS
LIMEQUERY_USERNAME = "XXX"  # <--- REPLACE THIS
LIMEQUERY_PASSWORD = "XXX"  # <--- REPLACE THIS

# --- Main Scraper Function ---
def scrape_limequery_lss_files(domain, username, password):
    """
    Automates logging into LimeSurvey and downloading LSS files for all surveys.

    Args:
        domain (str): The full URL to the LimeSurvey login page.
        username (str): Your LimeSurvey username.
        password (str): Your LimeSurvey password.
    """
    print(f"Starting LimeSurvey LSS download for domain: {domain}")

    # Set up Chrome options.
    # 'headless=True' runs the browser without a visible UI (good for servers).
    # Remove 'headless=True' if you want to see the browser actions.
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless") # Uncomment to run in headless mode
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080") # Set a fixed window size for consistent layout
    options.add_argument("--start-maximized") # Maximize window on start
    options.add_argument("--disable-gpu") # Required for some environments

    # Configure download preferences to try and automate downloads
    # Note: This attempts to set the default download directory, but browser security
    # may still prompt the user, especially for multiple files or different types.
    download_dir = os.path.join(os.getcwd(), "lss_downloads")
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)


    driver = None
    try:
        # Initialize the WebDriver
        # With modern Selenium, `executable_path` is no longer required as selenium-manager
        # automatically handles WebDriver downloads and setup.
        print("Initializing Chrome WebDriver (automatic management via selenium-manager)...")
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, 30) # Increased wait time for slow loading pages

        # 1. Navigate to the LimeSurvey login page
        print(f"Navigating to login page: {domain}")
        driver.get(domain)

        # 2. Input username and password
        print("Entering username and password...")
        user_field = wait.until(EC.presence_of_element_located((By.ID, "user")))
        user_field.send_keys(username)

        password_field = wait.until(EC.presence_of_element_located((By.ID, "password")))
        password_field.send_keys(password)

        # 3. Press the login button
        print("Clicking login button...")
        login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#loginform > div.row.login-submit.login-content > div > p > button")))
        login_button.click()

        # Wait for the dashboard to load (check for a common element on the dashboard)
        print("Waiting for dashboard to load...")
        try:
            wait.until(EC.url_contains("/admin/survey/sa/view")) # Common dashboard URL pattern
            print("Successfully logged in and reached dashboard.")
        except TimeoutException:
            print("Timeout waiting for dashboard URL. Checking for error messages or other indicators.")
            # Check for common login failure messages if dashboard URL isn't reached
            if "Invalid username or password" in driver.page_source:
                print("Login failed: Invalid username or password.")
                return
            else:
                print("Could not verify dashboard load. Proceeding with caution.")

        # --- Check for and close feature preview modal ---
        print("Checking for feature preview modal...")
        feature_preview_modal_xpath = "//*[@id='feature_preview_modal']/div/div" # This targets the modal-dialog/modal-content wrapper
        try:
            # Wait briefly for the modal to appear if it does
            # Ensure we're waiting for the outer modal structure before looking for content inside it
            feature_modal_container = wait.until(EC.presence_of_element_located((By.XPATH, feature_preview_modal_xpath)))
            print("Feature preview modal found. Attempting to close it.")
            
            close_button = None
            try:
                # Try the user-provided specific XPath first for the close button
                close_button = driver.find_element(By.XPATH, "//*[@id='feature_preview_modal']/div/div/div[1]/button")
                print("Found close button using specific XPath.")
            except NoSuchElementException:
                # Existing attempts (fallbacks)
                try:
                    # Try data-bs-dismiss="modal" first (Bootstrap 5)
                    close_button = feature_modal_container.find_element(By.CSS_SELECTOR, 'button[data-bs-dismiss="modal"]')
                except NoSuchElementException:
                    # Then try data-dismiss="modal" (Bootstrap 3/4)
                    try:
                        close_button = feature_modal_container.find_element(By.CSS_SELECTOR, 'button[data-dismiss="modal"]')
                    except NoSuchElementException:
                        # Then try common close classes like 'btn-close' or 'close'
                        try:
                            close_button = feature_modal_container.find_element(By.CSS_SELECTOR, '.btn-close')
                        except NoSuchElementException:
                            try:
                                close_button = feature_modal_container.find_element(By.CSS_SELECTOR, '.close')
                            except NoSuchElementException:
                                # As a last resort, try to find a button with "Close" text
                                try:
                                    close_button = feature_modal_container.find_element(By.XPATH, ".//button[contains(text(), 'Close')]")
                                except NoSuchElementException:
                                    pass # No button found

            if close_button:
                # Attempt to click directly, then fallback to JS if click intercepted
                try:
                    close_button.click()
                    print("Feature preview modal closed.")
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    print("Click intercepted for feature preview modal close button. Attempting JS click.")
                    driver.execute_script("arguments[0].click();", close_button)
                    print("Feature preview modal closed via JavaScript.")
                time.sleep(1) # Give time for modal to dismiss
            else:
                print("No obvious close button found within feature preview modal. Attempting to dismiss via JS.")
                # Fallback: execute JavaScript to hide the modal if no button is found
                driver.execute_script("arguments[0].style.display = 'none';", feature_modal_container)
                print("Feature preview modal dismissed via JavaScript.")
                time.sleep(1)
                
        except TimeoutException:
            print("Feature preview modal did not appear or timed out waiting for it (this is normal if it doesn't appear).")
        except Exception as e:
            print(f"Error handling feature preview modal: {e}")
        # --- End Check for and close feature preview modal ---


        # 4. On the dashboard, select 100 surveys per page
        print("Setting surveys per page to 100...")
        try:
            page_size_dropdown = wait.until(EC.presence_of_element_located((By.ID, "surveygrid--pageSize")))
            select = Select(page_size_dropdown)
            select.select_by_value("100") # Select by value '100'
            time.sleep(2) # Give some time for the page to refresh after changing page size
            print("Surveys per page set to 100.")
        except NoSuchElementException:
            print("Page size dropdown not found. It might not be available or selector has changed.")
        except Exception as e:
            print(f"Error setting page size: {e}")

        # 4.2 Select all surveys on the page
        print("Selecting all surveys on the current page...")
        try:
            select_all_checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='sid_all']")))
            if not select_all_checkbox.is_selected():
                select_all_checkbox.click()
            print("All surveys selected.")
            time.sleep(1) # Small pause after selection
        except NoSuchElementException:
            print("Select all checkbox not found. It might not be available or selector has changed.")
        except Exception as e:
            print(f"Error selecting all surveys: {e}")

        # Scroll to the bottom of the page before clicking the dropdown, to ensure visibility
        print("Scrolling to the bottom of the page...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1) # Give a moment for the scroll to complete

        # 4.3 Open the actions dropdown (using updated XPath)
        print("Opening survey actions dropdown (updated selector)...")
        try:
            actions_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='surveyListActions']/button/i")))
            actions_button.click()
            print("Actions dropdown opened.")
            time.sleep(2) # Increased pause for dropdown to fully render and items to be clickable
        except NoSuchElementException:
            print("Survey list actions button not found. It might not be available or selector has changed.")
        except Exception as e:
            print(f"Error opening actions dropdown: {e}")


        # 4.3 Choose "Export survey structure (*.lss)" (updated partial link text)
        print("Selecting 'Export survey structure (*.lss)' from dropdown...")
        try:
            # Using partial link text based on the provided outerHTML
            export_lss_link = wait.until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Survey structure (*.lss)")))
            export_lss_link.click()
            print("Export survey structure option clicked.")
            time.sleep(2) # Wait for modal to appear
            
        except NoSuchElementException:
            print("Export survey structure link not found. It might not be available or selector has changed.")
        except Exception as e:
            print(f"Error clicking export link: {e}")


        # 4.4 A popup will show. Click on the "Export" button/link within it.
        print("Waiting for first export modal and clicking Export button...")
        try:
            # Using the new, specific CSS selector for the Export button
            modal_export_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#massive-actions-modal-survey-grid-export-8 > div > div > div.modal-footer.modal-footer-buttons > a")))
            
            # Attempt to click directly, then fallback to JS if click intercepted/not interactable
            try:
                modal_export_button.click()
                print("Clicked Export button in the first modal.")
            except (ElementClickInterceptedException, ElementNotInteractableException):
                print("Click intercepted for first modal Export button. Attempting JS click.")
                driver.execute_script("arguments[0].click();", modal_export_button)
                print("Clicked Export button in the first modal via JavaScript.")

            time.sleep(3) # Wait for the next popup/download to initiate
        except TimeoutException:
            print("Timeout waiting for the first export modal button. Modal might not have appeared or selector is incorrect.")
        except NoSuchElementException:
            print("Export button in modal not found. It might be a different modal or selector has changed.")
        except Exception as e:
            print(f"An unexpected error occurred clicking first modal export button: {e}")


        # 4.5 Wait until another popup is shown. Then click on the download link.
        print("Waiting for second download modal and clicking download link...")
        try:
            # Using the new, specific CSS selector for the Download archive link
            download_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#massive-actions-modal-survey-grid-export-8 > div > div > div.modal-body > div.modal-body-text > div > a")))
            
            # Attempt to click directly, then fallback to JS if click intercepted/not interactable
            try:
                download_link.click()
                print("Clicked 'Download archive' link. Download should start.")
            except (ElementClickInterceptedException, ElementNotInteractableException):
                print("Click intercepted for second modal Download link. Attempting JS click.")
                driver.execute_script("arguments[0].click();", download_link)
                print("Clicked 'Download archive' link via JavaScript. Download should start.")

            time.sleep(5) # Give time for the download to initiate and complete
        except TimeoutException:
            print("Timeout waiting for the second download modal link. Modal might not have appeared or selector is incorrect.")
        except NoSuchElementException:
            print("Download link in second modal not found. It might be a different modal or selector has changed.")
        except Exception as e:
            print(f"An unexpected error occurred clicking second modal download link: {e}")


        print(f"Download process initiated. Files should be in: {download_dir}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if driver:
            print("Closing browser...")
            driver.quit()

# --- Execution ---
if __name__ == "__main__":
    # Ensure you have replaced the placeholder values for LIMEQUERY_USERNAME and LIMEQUERY_PASSWORD.
    if LIMEQUERY_USERNAME == "YOUR_USERNAME" or LIMEQUERY_PASSWORD == "YOUR_PASSWORD":
        print("WARNING: Please update LIMEQUERY_USERNAME and LIMEQUERY_PASSWORD in the script.")
        print("Exiting. You need to configure the script with your credentials.")
    else:
        # No webdriver_path needed for modern Selenium
        scrape_limequery_lss_files(LIMEQUERY_DOMAIN, LIMEQUERY_USERNAME, LIMEQUERY_PASSWORD)
